In [ ]:
from pasteur.kedro.ipython import *
register_kedro()

INFO     Kedro project Pasteur                                                                                           __init__.py:77
INFO     Defined global variable 'context', 'session', 'catalog' and 'pipelines'                                         __init__.py:78
INFO     Registered line magic 'run_viz'                                                                                 __init__.py:84
INFO     Registered line magic 'reload_kedro_mlflow'                                                                     __init__.py:84


In [ ]:
%load_ext autoreload
%autoreload 2
%load_ext line_profiler

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
from pasteur.metadata import Metadata

trn: pd.DataFrame = catalog.load("mimic_tab_admissions.trn.table")
table = catalog.load("mimic_tab_admissions.wrk.idx_table")

In [ ]:
attrs = trn["idx"].get_attributes()

In [ ]:
from copy import copy
from pasteur.synth.privbayes import rebalance_column

new_attrs = {}
for name, attr in attrs.items():
    cols = {}
    for col_name, col in attr.cols.items():
        cols[col_name] = rebalance_column(
            table[col_name], col, 0
        )

    new_attr = copy(attr)
    new_attr.update_cols(cols)
    new_attrs[name] = new_attr
new_attrs


{
    'birth_year': Attr[na=0,ukn=0]{},
    'dod': Attr[na=1,ukn=0]{'dod_year': <pasteur.synth.hierarchy.RebalancedColumn object at 0x7f8f28461240>, 'dod_week': <pasteur.synth.hierarchy.RebalancedColumn object at 0x7f8f28461270>, 'dod_day': <pasteur.synth.hierarchy.RebalancedColumn object at 0x7f8f284612d0>},
    'admittime': Attr[na=0,ukn=0]{'admittime_year': <pasteur.synth.hierarchy.RebalancedColumn object at 0x7f8f28461840>, 'admittime_week': <pasteur.synth.hierarchy.RebalancedColumn object at 0x7f8f284618a0>, 'admittime_day': <pasteur.synth.hierarchy.RebalancedColumn object at 0x7f8f28461900>, 'admittime_time': <pasteur.synth.hierarchy.RebalancedColumn object at 0x7f8f28461960>},
    'dischtime': Attr[na=0,ukn=0]{'dischtime_day': <pasteur.synth.hierarchy.RebalancedColumn object at 0x7f8f28461ba0>, 'dischtime_time': <pasteur.synth.hierarchy.RebalancedColumn object at 0x7f8f28461c00>},
    'deathtime': Attr[na=1,ukn=0]{'deathtime_day': <pasteur.synth.hierarchy.RebalancedColumn objec

In [ ]:
col_names = []  # col -> str name
groups = []  # col -> attribute parent
group_names = []  # attr (group) -> str name
heights = []  # col -> max height
common = []  # col -> common val number
domain = []  # col, height -> domain
for i, (an, a) in enumerate(new_attrs.items()):
    group_names.append(an)
    for n, c in a.cols.items():
        col_names.append(n)
        groups.append(i)
        heights.append(c.height)
        common.append(a.common)
        domain.append([c.get_domain(h) for h in range(c.height)])

print(domain)

[[33, 27, 21, 16, 12, 8, 5, 4, 2], [53, 45, 35, 27, 21, 16, 12, 8, 5, 4, 2], [8, 5, 4, 2], [32, 27, 21, 16, 12, 8, 5, 4, 2], [52, 45, 35, 27, 21, 16, 12, 8, 5, 4, 2], [7, 5, 4, 2], [24, 21, 16, 12, 8, 5, 4, 2], [64, 58, 45, 35, 27, 21, 16, 12, 8, 5, 4, 2], [24, 21, 16, 12, 8, 5, 4, 2], [65, 58, 45, 35, 27, 21, 16, 12, 8, 5, 4, 2], [25, 21, 16, 12, 8, 5, 4, 2], [9, 8, 5, 4, 2], [12, 8, 5, 4, 2], [14, 12, 8, 5, 4, 2], [3, 2], [2], [5, 4, 2], [8, 5, 4, 2], [2], [2]]


In [ ]:
from pasteur.synth.privbayes import EMPTY_PSET, add_to_pset

def maximal_parents(
    V: tuple[int], tau: float, P: dict[int, dict[int, int]] = {}
) -> list[tuple[int]]:
    """Given a set V containing hierarchical attributes (by int) and a tau
    score that is divided by the size of the domain, return a set of all
    possible combinations of attributes, such that if t > 1 there isn't an
    attribute that can be indexed in a higher level"""
    
    if not V:
        return [EMPTY_PSET]

    # Calculate domain manually to correct it based on common attrs
    dom = 1
    for g, a in P.items():
        l_dom = 1
        cmn = 0
        for x, h in a.items():
            cmn = common[x]  # same for x of a
            l_dom *= domain[x][h] - cmn
        dom *= l_dom + cmn

    if tau < dom:
        return []

    x = V[0]
    V = V[1:]

    S = []
    U = set()

    # Create copy of P for the version with x that has
    # a dict for x's group. Avoid deep copying
    P_x = P.copy()
    g = groups[x]
    gx = P.get(g, {}).copy()
    P_x[g] = gx

    for h in range(heights[x]):
        # Check domain is non-one when considering common attrs
        # Having multiple cols with domain 1 can skew exponential alg
        # into selecting columns from multi-col attributes
        if P.get(groups[x], None) and domain[x][h] == common[x] + 1:
            continue
        # Only changes local copy
        P_x[g][x] = h
        for z in maximal_parents(V, tau, P_x):
            if z not in U:
                U.add(z)
                S.append(add_to_pset(z, x, h))

    for z in maximal_parents(V, tau, P):
        if z not in U:
            S.append(z)

    return S

In [ ]:
V = (0, 2, 4, 6, 7, 10, 11, 13, 15, 16)
len(maximal_parents(V, 3000))


65184


In [ ]:
%timeit maximal_parents(V, 3000)

8.7 s ± 43.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [ ]:
%lprun -f maximal_parents maximal_parents(V, 1000)

Timer unit: 1e-06 s

Total time: 22.5815 s
File: /tmp/ipykernel_996423/2764480977.py
Function: maximal_parents at line 3

Line #      Hits         Time  Per Hit   % Time  Line Contents
     3                                           def maximal_parents(
     4                                               V: tuple[int], tau: float, P: dict[int, dict[int, int]] = {}
     5                                           ) -> list[tuple[int]]:
     6                                               """Given a set V containing hierarchical attributes (by int) and a tau
     7                                               score that is divided by the size of the domain, return a set of all
     8                                               possible combinations of attributes, such that if t > 1 there isn't an
     9                                               attribute that can be indexed in a higher level"""
    10                                               
    11   1295494     579617.0  

In [ ]:
A_dict = {}
for c in V:
    group = groups[c]
    if group in A_dict:
        A_dict[groups[c]].append(c)
    else:
        A_dict[groups[c]] = [c]
A = tuple(tuple(group) for group in A_dict.values())
A   

((0, 2), (4, 6), (7,), (10,), (11,), (13,), (15,), (16,))


In [ ]:
from pasteur.synth.privbayes import MAX_COLS, add_multiple_to_pset, EMPTY_PSET

empty_list = [-1 for _ in range(MAX_COLS)]
empty_activity = [False for _ in range(MAX_COLS)]

def maximal_parents_new(
    A: tuple[tuple[int]], tau: float, parent_dom: int = 1
) -> list[tuple[int]]:
    """Given a set V containing hierarchical attributes (by int) and a tau
    score that is divided by the size of the domain, return a set of all
    possible combinations of attributes, such that if t > 1 there isn't an
    attribute that can be indexed in a higher level"""

    a = A[0]
    a_n = len(a)
    g = groups[a[0]]
    cmn = common[a[0]]

    A = A[1:]
    S = []
    U = set()

    curr_attrs = list(empty_list)
    act_attrs = list(empty_activity)
    has_combs = True
    while has_combs:
        # Simple counter structure without iterators that will iterate over
        # all attribute height combinations
        i = 0
        carry = True
        while carry:
            carry = curr_attrs[i] == heights[a[i]] - 1 - (i != 0)
            act_attrs[i] = not carry

            if carry:
                curr_attrs[i] = -1
                i += 1
                # Detect overflow and break
                # Placing check on with condition would make it run every time
                if i == a_n:
                    has_combs = False
                    break
            else:
                curr_attrs[i] += 1

        # Find domain
        l_dom = parent_dom
        if has_combs:
            for i in range(a_n):
                h = curr_attrs[i]
                if h != -1:
                    dom = domain[a[i]][h]
                    l_dom *= dom - cmn
            l_dom += cmn
        # print(curr_attrs[:a_n], l_dom)

        # Run check to skip recursion
        if tau < l_dom:
            continue

        # Original algorithm ends by running another loop which doesn't include x
        # Conveniently, when i overflows and we run the last execution, curr_attrs
        # will consist of -1, which is equivalent
        if A:
            act_frozen = tuple(act_attrs)
            for z in maximal_parents_new(A, tau / l_dom):
                check = (act_frozen, z)
                if check not in U:
                    U.add(check)
                    S.append(add_multiple_to_pset(z, a, curr_attrs))
        else:
            S.append(add_multiple_to_pset(EMPTY_PSET, a, curr_attrs))

    return S

len(maximal_parents_new(A, 3000))

88424


In [ ]:
%timeit maximal_parents_new(A, 3000)

10.7 s ± 90 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [ ]:
# %lprun -f maximal_parents_new maximal_parents_new(A, 1000)

In [ ]:
from pasteur.synth.privbayes import add_multiple_to_pset, EMPTY_PSET, EMPTY_ACTIVE, EMPTY_LIST

def maximal_parents_new2(
    A: tuple[tuple[int]], tau: float, parent_dom: int = 1
) -> list[tuple[int]]:
    """Given a set V containing hierarchical attributes (by int) and a tau
    score that is divided by the size of the domain, return a set of all
    possible combinations of attributes, such that if t > 1 there isn't an
    attribute that can be indexed in a higher level"""

    if not A:
        return [EMPTY_PSET]

    a = A[0]
    a_n = len(a)
    x = a[0]
    g = groups[x]
    cmn = common[x]

    A = A[1:]
    S = []
    U = set()

    if a_n == 1:
        # Use specific implementation for single variables
        x = a[0]
        for h in range(heights[x]):
            # Find domain
            l_dom = parent_dom * (domain[x][h] - cmn) + cmn

            # Run check to skip recursion
            if tau < l_dom:
                continue

            for z in maximal_parents_new2(A, tau / l_dom):
                if z not in U:
                    U.add(z)
                    S.append(add_to_pset(z, x, h))
        
        for z in maximal_parents_new2(A, tau / parent_dom):
            if z not in U:
                S.append(z)
    else:
        # Compensating for multi-domain attrs is more complicated
        curr_attrs = list(EMPTY_LIST)
        act_attrs = list(EMPTY_ACTIVE)
        has_combs = True
        
        while has_combs:
            # Simple counter structure without iterators that will iterate over
            # all attribute height combinations
            for i in range(a_n):
                carry = curr_attrs[i] == heights[a[i]] - 1 - (i != 0)
                act_attrs[i] = not carry

                if carry:
                    curr_attrs[i] = -1
                    # Detect overflow and break
                    # Placing check on with condition would make it run every time
                    if i == a_n - 1:
                        has_combs = False
                else:
                    curr_attrs[i] += 1
                    break

            # Find domain
            l_dom = parent_dom
            if has_combs:
                for i in range(a_n):
                    h = curr_attrs[i]
                    if h != -1:
                        dom = domain[a[i]][h]
                        l_dom *= dom - cmn
                l_dom += cmn
            # print(curr_attrs[:a_n], l_dom)

            # Run check to skip recursion
            if tau < l_dom:
                continue

            # Original algorithm ends by running another loop which doesn't include x
            # Conveniently, when i overflows and we run the last execution, curr_attrs
            # will consist of -1, which is equivalent
            act_frozen = tuple(act_attrs)
            for z in maximal_parents_new2(A, tau / l_dom):
                check = (act_frozen, z)
                if check not in U:
                    U.add(check)
                    S.append(add_multiple_to_pset(z, a, curr_attrs))

    return S

len(maximal_parents_new2(A, 3000))
# maximal_parents_new2(A, 3000)[:10]


86454


In [ ]:
%timeit maximal_parents_new2(A, 3000)

3.87 s ± 10.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [ ]:
%lprun -f add_multiple_to_pset maximal_parents_new2(A, 300)

Timer unit: 1e-06 s

Total time: 0.574207 s
File: /mnt/ext/projects/pasteur/src/pasteur/synth/privbayes.py
Function: add_multiple_to_pset at line 96

Line #      Hits         Time  Per Hit   % Time  Line Contents
    96                                           def add_multiple_to_pset(s: tuple, x: list[int], h: list[int]):
    97                                               """Given parent set `s`, adds attributes `x` with heights `h`.
    98                                           
    99                                               `x` is checked for length, so `h` may be a larger array."""
   100                                           
   101    210502     143954.0      0.7     25.1      s = list(s)
   102                                           
   103    446275     164981.0      0.4     28.7      for i in range(len(x)):
   104    235773     102480.0      0.4     17.8          s[x[i]] = h[i]
   105    210502     162792.0      0.8     28.4      return tuple(s)